In [1]:
# -*- coding: utf-8 -*-
import re
import time
import random
import math
import numpy as np
import pandas as pd
import importlib
import logging
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
load_dotenv()

from src.tinderweb import TinderAutomator, Controller, SLEEP_MULTIPLIER
from src.data_interface import Backlog
from src.gpt3 import Gpt3, Allowance
from src.mymodels import TRAINED_MODELS

import urllib
from PIL import Image
import os
import re
import shutil
from matplotlib import pyplot as plt
import cv2

import configparser
config = configparser.ConfigParser()
config.read(os.environ["LOVEBOT_CONFIG"])

TLSv1.3


['D:/programming/lovebot/config.ini']

In [2]:
SLEEP_MULTIPLIER = int(config["DEFAULT"]["SleepTime"])
path_prefix = config['DEFAULT']["PathPrefix"]
scraping_folder_path = config['SCRAPING']["ScrapingFolder"]

In [3]:
logger = logging.getLogger('TA')
logger.setLevel(logging.INFO)
# create file handler which logs even debug messages
timestr = time.strftime("%Y%m%d-%H%M")
logging_file_name = f'logs/swiper_run_{timestr}.log'
fh = logging.FileHandler(path_prefix+logging_file_name, 'w', 'utf-8')
fh.setLevel(logging.INFO)
logger.addHandler(fh)

In [4]:
# initialize automator and models
ta = TinderAutomator(chromedata_path=config['DEFAULT']['ChromeDataPath'])


In [5]:
# initialize models
bikini_model = TRAINED_MODELS["bikini"](weights_path=config["MODELS"]["Bikini"])
like_model = TRAINED_MODELS["like"](weights_path=config["MODELS"]["Like"])

In [6]:
with Controller(ta) as controller:
    counter = 0
    reset_counter = 0
    target_count = int(config["SCRAPING"]["Count"])
    while (counter < target_count) and (reset_counter < int(config["SCRAPING"]["RetryCount"])):
        print(f"\n{counter}/{target_count}, errors {reset_counter}")
        try:
            controller.scrape(scraping_folder_path)
            preds, logitss = bikini_model.inference_pathlist(ta.current_profile.image_paths)
            ta.current_profile.has_bikini = "bikini" in preds
            # save image seperately if bikini
            bikini_index_list = [i for i, pred in enumerate(preds) if pred == 'bikini']
            bikini_paths = [ta.current_profile.image_paths[i] for i in bikini_index_list]
            for bikini_path in bikini_paths:
                shutil.copy2(bikini_path, scraping_folder_path+"/bikini/"+bikini_path.split("/")[-1])

            if not ta.current_profile.has_bikini:
                preds, logitss = like_model.inference_pathlist(ta.current_profile.image_paths, softmax=True)
                ta.current_profile.likescore = preds.count(like_model.classes[1]) / len(preds)
            
            print(ta.current_profile)
            #ta.current_profile.show_images()

            controller.choice()
            counter += 1
            reset_counter = 0
        except:
            reset_counter +=1
            print("error... resetting",reset_counter)
            ta.reset()


0/25, errors 0
on_scrape folder_name: D:/programming/lovebot/images
Opened card! Retrieved open_card_infos.
('Anne', '26', 3, '14f')

Anne, 26, 3km, 14f: Scraping 3 picture(s)...
error... resetting 1

0/25, errors 1
on_scrape folder_name: D:/programming/lovebot/images
Opened card! Retrieved open_card_infos.
('Diana', '33', 2, '91')

Diana, 33, 2km, 91: Scraping 9 picture(s)...
D:/programming/lovebot/images/Diana_33_2_91_0.jpg: no_bikini, tensor([[ 0.7826, -0.7942]])
D:/programming/lovebot/images/Diana_33_2_91_1.jpg: no_bikini, tensor([[ 1.1439, -1.1022]])
D:/programming/lovebot/images/Diana_33_2_91_2.jpg: no_bikini, tensor([[ 1.4668, -1.4340]])
D:/programming/lovebot/images/Diana_33_2_91_3.jpg: no_bikini, tensor([[ 0.5835, -0.2565]])
D:/programming/lovebot/images/Diana_33_2_91_4.jpg: no_bikini, tensor([[ 1.0368, -0.5691]])
D:/programming/lovebot/images/Diana_33_2_91_5.jpg: no_bikini, tensor([[ 0.7607, -0.6547]])
D:/programming/lovebot/images/Diana_33_2_91_6.jpg: no_bikini, tensor([[ 1

d:\programming\lovebot\src\mymodels.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(logits)


D:/programming/lovebot/images/Diana_33_2_91_2.jpg: dislike, tensor([[0.7480, 0.2520]])
D:/programming/lovebot/images/Diana_33_2_91_3.jpg: like, tensor([[0.3865, 0.6135]])
D:/programming/lovebot/images/Diana_33_2_91_4.jpg: like, tensor([[0.3791, 0.6209]])
D:/programming/lovebot/images/Diana_33_2_91_5.jpg: dislike, tensor([[0.8208, 0.1792]])
D:/programming/lovebot/images/Diana_33_2_91_6.jpg: dislike, tensor([[0.6526, 0.3474]])
D:/programming/lovebot/images/Diana_33_2_91_7.jpg: dislike, tensor([[0.8010, 0.1990]])
D:/programming/lovebot/images/Diana_33_2_91_8.jpg: dislike, tensor([[0.7737, 0.2263]])
Diana_33_2_91

pic_count: 9
bikini pic: False
likescore: 0.4444444444444444

choice: None
Evaluated choice to left because of likescore
Swiping left. Closing card!
Swiping!

1/25, errors 0
on_scrape folder_name: D:/programming/lovebot/images
Opened card! Retrieved open_card_infos.
('Natalia', '24', 3, '86')

Natalia, 24, 3km, 86: Scraping 7 picture(s)...
D:/programming/lovebot/images/Natalia_24